# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

# Short introduction

At the beginning of the assignment I had an idea about using CBOW model as a main solution algorithm. It was about using CBOW weights in order to predict the misspelled word based on its context words and then comparing it to correction candidates. However, the idea failed miserably not even reaching the baseline level of Norvig's super simple frequency-based implementation. I spent around 3 full days trying to make it work(some solutions were crazy enough to send me to the asylum) and had quite a lot of fun with it, but then just used a BERT model instead and got a much better accuracy after writing code for just a couple of hours. I am not sure if you are interested in an awful word2vec implementation, but I will share it anyway)

# Baseline N-gram solution

In [ ]:
import re
import collections
from itertools import islice
from typing import List, Set, Dict

class NGramSpellingCorrector:
    def __init__(self, corpus_file, n):
        self.n = n 
        self.words = self.load_corpus(corpus_file)
        self.vocab = set(self.words)
        self.word_freqs = collections.Counter(self.words)
        self.ngrams = self.build_ngrams(self.words, n)

    def load_corpus(self, corpus_file):
        with open(corpus_file, "r", encoding="utf-8") as f:
            text = f.read().lower()
        return re.findall(r'\w+', text)

    def build_ngrams(self, words, n):
        ngram_model = collections.defaultdict(set)
        for i in range(len(words) - n + 1):
            context = tuple(words[i:i + n - 1]) 
            next_word = words[i + n - 1]
            ngram_model[context].add(next_word)
        return ngram_model

    def known(self, words):
        return {w for w in words if w in self.vocab}

    def edits1(self, word):
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = {L + R[1:] for L, R in splits if R}
        transposes = {L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1}
        replaces = {L + c + R[1:] for L, R in splits if R for c in letters}
        inserts = {L + c + R for L, R in splits for c in letters}
        return set(deletes | transposes | replaces | inserts)

    def edits2(self, word):
        return {e2 for e1 in self.edits1(word) for e2 in self.edits1(e1)}

    def candidates(self, word, context):
        return (self.known([word]) or
                self.known(self.edits1(word)) or
                self.known(self.edits2(word)) or
                {word})

    def probability(self, word):
        return self.word_freqs[word] / sum(self.word_freqs.values())

    def correct(self, word, context):
        candidates = self.candidates(word, context)
        
        if not context:
            return max(candidates, key=self.probability)
        
        context_tuple = tuple(context[-(self.n - 1):])
        possible_words = self.ngrams.get(context_tuple, set())
        valid_candidates = candidates & possible_words
    
        return max(valid_candidates or candidates, key=self.probability)


In [ ]:
def load_eval_dataset(file_path="eval.txt"):
    data = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split('|')
            if len(parts) != 3:
                continue
            
            sentence, misspelled, correct = parts
            words = sentence.split()
            
            if misspelled in words:
                words.remove(misspelled)
            
            data.append({
                'sentence': sentence,
                'misspelled': misspelled,
                'correct': correct,
                'context': words
            })
    
    return data

def evaluate_model(model, eval_data):
    correct = 0
    total = 0
    
    for case in eval_data:
        predicted = model.correct(case['misspelled'], case['context'])
        print(f"Sentence: {case['sentence']} | Predicted: {predicted} | Correct: {case['correct']}")
        
        total += 1
        if predicted == case['correct']:
            correct += 1

    overall_accuracy = correct / total if total > 0 else 0
    
    print(f"\n{' Overall Accuracy ':-^40}")
    print(f"{correct}/{total} ({overall_accuracy:.2%})\n")

if __name__ == "__main__":
    corpus_path = "/kaggle/input/big-text/big.txt"
    eval_file_path = "/kaggle/input/eval-ambiguous/eval.txt"

    corrector = NGramSpellingCorrector(corpus_path, n=3)
    eval_data = load_eval_dataset(eval_file_path)
    evaluate_model(corrector, eval_data)


Sentence: The car drve fast on the highway. | Predicted: drove | Correct: drove
Sentence: She saw a brid flying in the sky. | Predicted: rid | Correct: bird
Sentence: He used a pencl to write notes. | Predicted: pencil | Correct: pencil
Sentence: They found a brwn puppy in the park. | Predicted: brown | Correct: brown
Sentence: My friend brke his phone screen. | Predicted: broke | Correct: broke
Sentence: The dog dug a hol in the yard. | Predicted: how | Correct: hole
Sentence: She wore a yelow dress to the party. | Predicted: below | Correct: yellow
Sentence: I heard a lod noise outside. | Predicted: old | Correct: loud
Sentence: The old man had a wlk in the garden. | Predicted: walk | Correct: walk
Sentence: We took a bote to the island. | Predicted: bone | Correct: boat
Sentence: His glss broke when it fell. | Predicted: glass | Correct: glass
Sentence: She drew a smal picture in her notebook. | Predicted: small | Correct: small
Sentence: The fire brnt the wood completely. | Predict

# This solution did not work

This is based on the mentioned idea about calculating CBOW embeddings for predicting the most alike corrections. Since the number of context words varies significantly, we cannot just predict the embedding well, making it necessary to get the mean embedding of the context words. Probably it is possible to make it work this way with adding special tokens instead of missing words(for example when the misspelled word is located at the end of a sentence), but I did not get good results while trying to implement it, mainly because it takes a lot of time to train meaninful embeddings while making little changes. (less than 50 epochs is useless on this dataset, more than 50 epochs takes more than 3 hous to train)

In [ ]:
import os
from gensim.models import Word2Vec
from gensim.utils import tokenize
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re

def load_dataset(file_path="/kaggle/input/big-text/big.txt"):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', text)
            for sentence in sentences:
                yield list(tokenize(sentence.lower(), deacc=True))
    except UnicodeDecodeError:
        print(f"Error: Unable to read {file_path} as a text file.")

def initialize_model(sentences, embedding_size=300, window=5, min_count=20, workers=8):
    model = Word2Vec(
        vector_size=embedding_size,
        window=window,
        min_count=min_count,
        sg=0,
        workers=workers
    )
    model.build_vocab(sentences)
    return model

def train_model_incrementally(model, sentences, epochs=10):
    print(f"Training model...")
    total_examples = len(sentences)
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train(sentences, total_examples=total_examples, epochs=1)
    return model

def generate_candidates(word, edit_distance=2):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    operations = {
        'deletes': [L + R[1:] for L, R in splits if R],
        'inserts': [L + c + R for L, R in splits for c in letters],
        'replaces': [L + c + R[1:] for L, R in splits if R for c in letters]
    }
    candidates = set().union(*operations.values())
    if edit_distance == 2:
        candidates.update([c2 for c1 in candidates for c2 in generate_candidates(c1, 1)])
    return candidates

def correct_spelling(model, word, context_words, verbose=True):
    if verbose:
        print(f"Correcting spelling for: {word}")
        print(f"Context: {context_words}")

    candidates = generate_candidates(word)
    if verbose:
        print(f"{len(candidates)} candidates found")

    valid_candidates = [c for c in candidates if c in model.wv]
    if verbose:
        print(f"Valid corrections: {valid_candidates}")

    if not valid_candidates:
        if verbose:
            print("No valid candidates")
        return word

    context_embeddings = [model.wv[w] for w in context_words if w in model.wv]
    if not context_embeddings:
        if verbose:
            print("No valid context, something went wrong")
        return word
        
    context_vector = np.mean(context_embeddings, axis=0)

    output_weights = model.syn1neg

    candidate_indices = [model.wv.key_to_index[c] for c in valid_candidates]
    candidate_scores = np.dot(context_vector, output_weights[candidate_indices].T)
    
    best_idx = np.argmax(candidate_scores)
    best_candidate = valid_candidates[best_idx]
    
    if verbose:
        print(f"Predicted scores:")
        for c, score in zip(valid_candidates, candidate_scores):
            print(f"{c}: {score}")
        print(f"Best correction: '{best_candidate}'")

    return best_candidate

def load_eval_dataset(file_path="eval.txt"):
    data = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split('|')
            if len(parts) != 3:
                continue
            
            sentence, misspelled, correct = parts
            words = sentence.split()
            
            if misspelled in words:
                words.remove(misspelled)
            
            data.append({
                'sentence': sentence,
                'misspelled': misspelled,
                'correct': correct,
                'context': words
            })
    
    return data

def evaluate_model(model, eval_data):
    correct = 0
    total = 0
    
    for case in eval_data:
        predicted = correct_spelling(model, case['misspelled'], case['context'], verbose=False)
        print(f"Sentence: {case['sentence']} | Predicted: {predicted} | Correct: {case['correct']}")
        
        total += 1
        
        if predicted == case['correct']:
            correct += 1

    overall_accuracy = correct / total if total > 0 else 0
    
    print(f"\n{' Overall Accuracy '}")
    print(f"{correct}/{total} ({overall_accuracy})\n")

In [ ]:
if __name__ == "__main__":
    print("Loading dataset...")
    sentences = list(load_dataset(file_path="/kaggle/input/smaller_train/train(1).txt"))
    print(f"Loaded {len(sentences)} sentences for training")

    model = initialize_model(sentences)

    epochs = 50
    model = train_model_incrementally(model, sentences, epochs=epochs)
    model.save("cbow_model_final.bin")
    print("Final model saved as cbow_model_final.bin")

    print("Loading eval dataset...")
    eval_data = load_eval_dataset("/kaggle/input/eval-ambiguous/eval.txt")
    print(f"Loaded {len(eval_data)} evaluation cases")
    print("Evaluating model...")
    evaluate_model(model, eval_data)

Loading dataset...
Loaded 12013910 sentences for training
Training model...
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
Saving final model...
Final model saved as cbow_model_final.bin
Loading evaluation dataset...
Loaded 97 evaluation cases
Evaluating model...
Sentence: The car drve fast on the highway. | Predicted: driver | Correct: drove
Sentence: She saw a brid flying in the sky. | Predicted: bird | Correct: bird
Sentence: He used a pencl to writ

In [ ]:
print("Sanity check - Most similar to king:")
print(model.wv.most_similar("king", topn=5))

print("Sanity check - Most similar to bread:")
print(model.wv.most_similar("bread", topn=5))

# Solution that worked well

This solution works in a similar matter to CBOW. The difference is that the pre-trained BERT model is used to predict the masked misspelled word and compare the embeddings with candidate corrections. It still works quite fast, keeping the correction system efficient enough to use.

It works much better since it actually understand which words are better fit based on sentence context.

In [ ]:
import re
import torch
import string
from collections import Counter
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
model.eval()

def words(text):
    return re.findall(r'\w+', text.lower())

def load_corpus(file_path='/kaggle/input/plain-text-wikipedia-simpleenglish/AllCombined.txt'):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def word_probabilities(word_counts):
    total_words = sum(word_counts.values())
    return {word: count / total_words for word, count in word_counts.items()}


# Candidate generation function was brazenly stolen from Norvig's implementation
def edits1(word):
    letters = string.ascii_lowercase
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

def known(words, word_counts):
    return {word for word in words if word in word_counts}

def mask_and_predict(sentence, target_index):
    words = sentence.split()
    words[target_index] = "[MASK]"
    masked_sentence = " ".join(words)

    tokens = tokenizer(masked_sentence, return_tensors="pt")
    mask_index = tokens.input_ids[0].tolist().index(tokenizer.mask_token_id)

    with torch.no_grad():
        outputs = model(**tokens)
    predictions = outputs.logits[0, mask_index]

    top_tokens = torch.topk(predictions, 10).indices.tolist()
    predicted_words = [tokenizer.decode([token]).strip() for token in top_tokens]

    return predicted_words

def correct_spelling(sentence, target_index, word_counts):
    bert_predictions = mask_and_predict(sentence, target_index)
    
    print("\nTop BERT Predictions:")
    for i, word in enumerate(bert_predictions):
        print(f"{i + 1}. {word}")

    misspelled_word = sentence.split()[target_index]

    candidates = known(edits1(misspelled_word), word_counts) or \
                 known(edits2(misspelled_word), word_counts) or \
                 {misspelled_word}

    print(f"\nCandidate Words: {candidates}")

    for word in bert_predictions:
        if word in candidates:
            return word

    return max(candidates, key=word_counts.get, default=misspelled_word)

if __name__ == '__main__':
    corpus = load_corpus('/kaggle/input/plain-text-wikipedia-simpleenglish/AllCombined.txt')
    word_counts = Counter(words(corpus))
    word_probs = word_probabilities(word_counts)

    sentence = "I want to by a new laptop."
    target_index = 3

    corrected_word = correct_spelling(sentence, target_index, word_counts)
    print(f"Final Corrected Word: {corrected_word}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Top BERT Predictions:
1. buy
2. get
3. have
4. build
5. purchase
6. open
7. find
8. start
9. make
10. order

Candidate Words: {'bn', 'nby', 'boy', 'sy', 'bay', 'bc', 'bly', 'jy', 'pby', 'be', 'byn', 'my', 'ay', 'eby', 'py', 'bya', 'yy', 'bo', 'bm', 'bw', 'xy', 'bby', 'bj', 'oy', 'bv', 'bl', 'ty', 'byk', 'dy', 'bi', 'bp', 'bk', 'bq', 'uy', 'cy', 'byl', 'yb', 'bye', 'ey', 'wy', 'byu', 'ly', 'fy', 'byp', 'ky', 'aby', 'hy', 'bt', 'ry', 'bf', 'by', 'bu', 'byg', 'bd', 'br', 'bb', 'bx', 'zy', 'ny', 'y', 'byo', 'ba', 'byb', 'bz', 'bs', 'b', 'bh', 'bey', 'iy', 'bys', 'bg', 'bry', 'vy', 'gy', 'buy'}

Final Corrected Word: buy


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

As for datasets I used concatenated wikipedia snapshot + reddit dialogues for CBOW model and just wikipedia-small for vocabulary of BERT-based implementation. It is actually not too bad to use datasets aside from official vocabularies or other verified resources since the model ususally does not assign high probabilities to the words misspelled by users by mistake.

Because we are looking for corrections in the context-aware space, it does not matter much how much weight to assign to edit1 and edit2, since situations when we have two value-similar words which are 1 letter distant from each other are super rare and it is not required to pay a lot of attention to such cases. 

The top-10 BERT candidates approach was chosen solely imperically to give better results and still keep the system stable and avoid ilogical word sequences.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

Evaluation dataset was generated by an LLM just because those small datasets that I could find on the internet lacked ambiguity in terms of possible word meanings, which is a key parameter when testing a spelling corrector. It contains 97 different sentences with multiple word meanings.

The correction speed for BERT model is about 100 times slower compared to Norvig's solution, but still not too bad and is limited by approximately 0.1 seconds making it possible to use for usual purposes. Norvig's solution is simple string operations and takes about 0.001 per word. To make the system faster, we could use DistillBert or some similar model

In [ ]:
def load_eval_dataset(file_path="eval.txt"):
    data = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split('|')
            if len(parts) != 3:
                continue
            
            sentence, misspelled, correct = parts
            words = sentence.split()
            
            if misspelled in words:
                words.remove(misspelled)
            
            data.append({
                'sentence': sentence,
                'misspelled': misspelled,
                'correct': correct,
                'context': words
            })
    
    return data

def evaluate_bert_model(eval_data, word_counts):
    correct_total = 0
    total = 0

    for case in eval_data:
        sentence = case['sentence']
        correct_word = case['correct']
        
        words = sentence.split()
        try:
            target_index = words.index(case['misspelled'])
        except ValueError:
            print(f"Skipping: {sentence} (Misspelled word not found)")
            continue

        predicted = correct_spelling(sentence, target_index, word_counts)

        print(f"Sentence: {sentence} | Predicted: {predicted} | Correct: {correct_word}")

        total += 1
        if predicted == correct_word:
            correct_total += 1

    overall_accuracy = correct_total / total if total > 0 else 0
    print(f"\n{' Overall Accuracy ':-^40}")
    print(f"{correct_total}/{total} ({overall_accuracy:.2%})\n")

if __name__ == '__main__':
    eval_data = load_eval_dataset('/kaggle/input/eval-ambiguous/eval.txt')
    evaluate_bert_model(eval_data, word_counts)



Top BERT Predictions:
1. was
2. moved
3. drove
4. went
5. turned
6. ran
7. stopped
8. spun
9. sped
10. got

Candidate Words: {'erve', 'drv', 'dive', 'drse', 'drue', 'drave', 'orve', 'drive', 'irve', 'arve', 'drev', 'drove', 'dre', 'dave', 'duve', 'dove', 'deve', 'dree', 'dve'}
Sentence: The car drve fast on the highway. | Predicted: drove | Correct: drove

Top BERT Predictions:
1. helicopter
2. bird
3. plane
4. figure
5. shadow
6. ufo
7. ship
8. comet
9. shape
10. cloud

Candidate Words: {'brd', 'brid', 'barid', 'arid', 'frid', 'drid', 'brio', 'rid', 'brie', 'bri', 'briz', 'bred', 'brind', 'bric', 'grid', 'krid', 'brik', 'braid', 'brix', 'bris', 'brit', 'brij', 'bid', 'brih', 'bird', 'bria', 'bridi', 'brim', 'brin', 'briw', 'brig', 'bride', 'bruid', 'brod', 'urid', 'brad'}
Sentence: She saw a brid flying in the sky. | Predicted: bird | Correct: bird

Top BERT Predictions:
1. pen
2. pencil
3. notebook
4. cane
5. comb
6. computer
7. machine
8. stick
9. book
10. brush

Candidate Words: {

In [ ]:
import re
from collections import Counter

def words(text):
    return re.findall(r'\w+', text.lower())

def load_corpus(file_path='/kaggle/input/big-text/big.txt'):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def word_probabilities(word_counts):
    total_words = sum(word_counts.values())
    return {word: count / total_words for word, count in word_counts.items()}

def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

def known(words, word_counts):
    return {word for word in words if word in word_counts}

def correct(word, word_counts, word_probs):
    candidates = (known([word], word_counts) or
                  known(edits1(word), word_counts) or
                  known(edits2(word), word_counts) or
                  [word])
    return max(candidates, key=word_probs.get)

if __name__ == '__main__':
    corpus = load_corpus('/kaggle/input/big-text/big.txt')
    word_counts = Counter(words(corpus))
    word_probs = word_probabilities(word_counts)
    
    misspelled_word = 'speling'
    correction = correct(misspelled_word, word_counts, word_probs)
    print(f"Correction for '{misspelled_word}': {correction}")


Correction for 'speling': spelling


In [ ]:
eval_data = load_eval_dataset('/kaggle/input/eval-ambiguous/eval.txt')

def evaluate_norvig_model(eval_data):
    correct_total = 0
    total = 0

    for case in eval_data:
        predicted = correct(case['misspelled'], word_counts, word_probs)
        print(f"Sentence: {case['sentence']} | Predicted: {predicted} | Correct: {case['correct']}")

        total += 1

        if predicted == case['correct']:
            correct_total += 1

    overall_accuracy = correct_total / total if total > 0 else 0

    print(f"\n{' Overall Accuracy ':-^40}")
    print(f"{correct_total}/{total} ({overall_accuracy:.2%})\n")

evaluate_norvig_model(eval_data)


Sentence: The car drve fast on the highway. | Predicted: drove | Correct: drove
Sentence: She saw a brid flying in the sky. | Predicted: rid | Correct: bird
Sentence: He used a pencl to write notes. | Predicted: pencil | Correct: pencil
Sentence: They found a brwn puppy in the park. | Predicted: brown | Correct: brown
Sentence: My friend brke his phone screen. | Predicted: broke | Correct: broke
Sentence: The dog dug a hol in the yard. | Predicted: how | Correct: hole
Sentence: She wore a yelow dress to the party. | Predicted: below | Correct: yellow
Sentence: I heard a lod noise outside. | Predicted: old | Correct: loud
Sentence: The old man had a wlk in the garden. | Predicted: walk | Correct: walk
Sentence: We took a bote to the island. | Predicted: bone | Correct: boat
Sentence: His glss broke when it fell. | Predicted: glass | Correct: glass
Sentence: She drew a smal picture in her notebook. | Predicted: small | Correct: small
Sentence: The fire brnt the wood completely. | Predict

As it can be seen, BERT-based solution makes much better and explainable solutions resulting in 75% of accuracy and can be further finetuned for specific needs for example assigning higher probabilities to words frequently typed by a user, while baseline Norvig's solution has only 45% of accuracy and quite limited by it, since it cannot get the semantics and context of a typed sentence.

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)